In [3]:
"""
个性化推荐的实例用法
"""

import json

from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_openai import ChatOpenAI

from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.store.memory import BaseStore, InMemoryStore

from dotenv import load_dotenv

load_dotenv()

recommendation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个乐于助人的推荐引擎。根据用户资料，提供个性化的产品推荐。"),
        ("human", "{user_profile_summary}"),
    ]
)
recommendation_chain = (
    recommendation_prompt
    | ChatOpenAI(model="Qwen/Qwen2.5-7B-Instruct")
    | (lambda x: {"messages": [AIMessage(content=x.content)]})
)


def recommend_products(state: MessagesState, config: RunnableConfig, store: BaseStore):
    user_id = config["configurable"]["user_id"]
    namespace = ("user_profiles", user_id)
    user_profile_record = store.get(namespace, "profile")
    user_profile = user_profile_record.value if user_profile_record else {}
    user_profile_summary = format_user_profile_summary(user_profile)
    result = recommendation_chain.invoke({"user_profile_summary": user_profile_summary})
    return result


def format_user_profile_summary(user_profile: dict) -> str:
    name = user_profile.get("preferred_name", "用户")
    categories = ", ".join(user_profile.get("preferred_product_categories", ["产品"]))
    return f"用户名为 {name}。他们偏好的产品类别是：{categories}"


def extract_preference_updates(state: MessagesState) -> dict:
    latest_message_content = state["messages"][-2].content
    extraction_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "从用户消息中提取用户的产品类别偏好。以 JSON 字典形式返回，外层不要包裹 '''json'''，键为 'preferred_product_categories'，值为类别列表。如果没有表达偏好，则返回一个空字典。",
            ),
            ("human", "{user_message}"),
        ]
    )
    extraction_chain = extraction_prompt | ChatOpenAI(model="Qwen/Qwen2.5-7B-Instruct")
    preferences_json = extraction_chain.invoke({"user_message": latest_message_content})
    try:
        preferences = json.loads(preferences_json.content)
        return preferences
    except json.JSONDecodeError:
        return {}


def update_user_profile_node(
    state: MessagesState, config: RunnableConfig, store: BaseStore
):
    user_id = config["configurable"]["user_id"]
    namespace = ("user_profiles", user_id)
    user_profile_record = store.get(namespace, "profile")
    user_profile = user_profile_record.value if user_profile_record else {}
    preference_updates = extract_preference_updates(state)
    updated_profile = user_profile.copy()
    if "preferred_product_categories" in preference_updates:
        updated_profile["preferred_product_categories"] = list(
            set(
                updated_profile.get("preferred_product_categories", [])
                + preference_updates["preferred_product_categories"]
            )
        )

    store.put(namespace, "profile", updated_profile)

    return {}


memory_store = InMemoryStore()

builder = StateGraph(MessagesState)
builder.add_node("recommend_products", recommend_products)
builder.add_node("update_profile", update_user_profile_node)
builder.add_edge(START, "recommend_products")
builder.add_edge("recommend_products", "update_profile")
builder.add_edge("update_profile", END)

graph = builder.compile(store=memory_store)

user_id = "user_123"
memory_store.put(
    ("user_profiles", user_id),
    "profile",
    {
        "preferred_name": "张三",
        "preferred_product_categories": ["电子产品", "书籍"],
    },
)

config = {"configurable": {"user_id": user_id}}
result = graph.invoke({"messages": [HumanMessage(content="你好")]}, config=config)
print("初始推荐：")
print(result["messages"][-1].content)
print("=" * 50)

user_message = "我最近对户外装备和运动鞋很感兴趣。"
result = graph.invoke(
    {"messages": result["messages"] + [HumanMessage(content=user_message)]},
    config=config,
)

updated_profile = memory_store.get(("user_profiles", user_id), "profile").value
print("更新后的用户资料：")
print(json.dumps(updated_profile, ensure_ascii=False, indent=2))
print("=" * 50)

result = graph.invoke({"messages": [HumanMessage(content="我又来了")]}, config=config)
print("基于更新后资料的推荐：")
print(result["messages"][-1].content)

初始推荐：
根据你提供的用户资料，以下是针对张三用户偏好的产品推荐：

电子产品推荐：
1. 华为Mate40 Pro：这款手机配备了麒麟9000芯片，拥有出色的性能和强大的拍照功能。
2. 小米电视6：该电视产品拥有超高的色彩还原度和清晰度，提升了画面的细腻度。此外，小米电视6还配备有高端的音质系统，让观看电影和做游戏都有更好的体验。
3. 一加9 Pro：一加手机一直以优化的用户体验和流畅的操作系统受到用户的欢迎，这款手机更是具有强大的功能和优质的性能。

书籍推荐：
1. 《安徒生童话选》：收录了安徒生经典童话故事，包括《丑小鸭》《海的女儿》《卖火柴的小女孩》等，适合各个年龄段的读者，不仅能培养小朋友的艺术素养，还能触动儿童爱家人、愿爱人之情怀。
2. 《废都》：中国当代著名作家贾平凹的作品，通过描绘一个充满生机的城市，探讨现代人面临的困惑和矛盾。
3. 《活着》：余华的代表小说作品，通过主人公的视角展现命运的无常与人生的苦难，同时也揭示了人与人之间真挚的情感纽带。
更新后的用户资料：
{
  "preferred_name": "张三",
  "preferred_product_categories": [
    "运动鞋",
    "户外装备",
    "电子产品",
    "书籍"
  ]
}
基于更新后资料的推荐：
根据您的个人信息，我为您推荐以下几种产品：

1. 运动鞋：adidas NMD XR1 ，这款鞋子采用加强的 Primeblue 重新设计，鞋身上饰有多款动物印刷，复古气息浓郁，造型别具一格。适合参加各种体育运动，例如篮球，足球，瑜伽等。

2. 户外装备：充气睡袋，让您在帐篷或者休息的地方能够得到充足的休息，特别适合长途徒步或者开车出行享受户外运动。

3. 电子产品：华为Mate 40 Pro 5G，华为的这款新机型拥有独特的环形摄像头，独特的CMOS，支持5G网络以及强大的处理器。在户外的时候，可以使用这款手机拍摄视频，随后利用5G网络及时将视频分享给好友。

4. 书籍：《人类极地探险史》这本书以详实的数据和真实的故事，为我们揭秘了人类极地探险史上的种种重要事件和发现。适合对人类探险历史感兴趣的朋友。
